## Tranformar datase de IDS para utilizar apenas dimensão economica

In [1]:
import pandas as pd
from lxml import html
import requests
import zipfile
import io
import os
from pathlib import Path
import csv

In [2]:
df = pd.read_csv("http://ftp.dadosabertos.ans.gov.br/FTP/PDA/historico_idss/historico_idss.csv",sep=";",encoding="ISO-8859-1")

In [3]:
df.columns

Index(['Registro', 'CNPJ', 'Razao_Social', 'Modalidade', 'IDSS_2009_2008',
       'IDSS_2010_2009', 'IDSS_2011_2010', 'IDAS_2011_2010', 'IDEO_2011_2010',
       'IDEF_2011_2010', 'IDSB_2011_2010', 'IDSS_2012_2011', 'IDAS_2012_2011',
       'IDEO_2012_2011', 'IDEF_2012_2011', 'IDSB_2012_2011', 'IDSS_2013_2012',
       'IDAS_2013_2012', 'IDEO_2013_2012', 'IDEF_2013_2012', 'IDSB_2013_2012',
       'IDSS_2014_2013', 'IDAS_2014_2013', 'IDEO_2014_2013', 'IDEF_2014_2013',
       'IDSB_2014_2013', 'IDSS_2015_2014', 'IDAS_2015_2014', 'IDEO_2015_2014',
       'IDEF_2015_2014', 'IDSB_2015_2014', 'IDSS_2016_2015', 'IDQS_2016_2015',
       'IDGA_2016_2015', 'IDSM_2016_2015', 'IDGR_2016_2015', 'IDSS_2017_2016',
       'IDQS_2017_2016', 'IDGA_2017_2016', 'IDSM_2017_2016', 'IDGR_2017_2016',
       'IDSS_2018_2017', 'IDQS_2018_2017', 'IDGA_2018_2017', 'IDSM_2018_2017',
       'IDGR_2018_2017'],
      dtype='object')

In [4]:
df = df[['Registro','Razao_Social','Modalidade','IDSM_2018_2017']]

## Filtrando para obter apenas cooperativas

In [5]:
df_cooperativas = df[df['Modalidade'] == "Cooperativa Médica"]

In [6]:
len(df_cooperativas)

342

In [7]:
df_cooperativas.dropna(inplace=True) 

/home/leticia/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [8]:
len(df_cooperativas)

342

In [9]:
df_cooperativas.head()

,Registro,Razao_Social,Modalidade,IDSM_2018_2017
14,300136,UNIMED LITORAL SUL/RS - COOPERATIVA MÉDICA LTDA.,Cooperativa Médica,"0,6030"
18,300713,UNIMED DE ASSIS COOPERATIVA DE TRABALHO MÉDICO,Cooperativa Médica,"0,9640"
21,300870,UNIMED DO BRASIL - CONF. NACIONAL DAS COOPERAT...,Cooperativa Médica,"0,5714"
25,301060,UNIMED ITUIUTABA COOPERATIVA TRABALHO MÉDICO L...,Cooperativa Médica,"0,9914"
26,301124,UNIMED GURUPI COOPERATIVA DE TRABALHO MÉDICO,Cooperativa Médica,"0,6660"


In [10]:
def converter_para_float(x):
    try:
        return float(x.replace(".","").replace(",","."))
    except:
        return 0

### Tranformando índices em variáveis do tipo float

In [11]:
colunas_indices = ['IDSM_2018_2017']
for coluna in colunas_indices:
    df_cooperativas[coluna] = df_cooperativas[coluna].apply(lambda x: converter_para_float(x))

/home/leticia/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
df_cooperativas.head()

,Registro,Razao_Social,Modalidade,IDSM_2018_2017
14,300136,UNIMED LITORAL SUL/RS - COOPERATIVA MÉDICA LTDA.,Cooperativa Médica,0.6030
18,300713,UNIMED DE ASSIS COOPERATIVA DE TRABALHO MÉDICO,Cooperativa Médica,0.9640
21,300870,UNIMED DO BRASIL - CONF. NACIONAL DAS COOPERAT...,Cooperativa Médica,0.5714
25,301060,UNIMED ITUIUTABA COOPERATIVA TRABALHO MÉDICO L...,Cooperativa Médica,0.9914
26,301124,UNIMED GURUPI COOPERATIVA DE TRABALHO MÉDICO,Cooperativa Médica,0.6660


## Filtrando para pegar as cooperativas com o IDMS igual a 1

In [13]:
df_cooperativas = df_cooperativas[df_cooperativas['IDSM_2018_2017']==1]

In [14]:
# df_cooperativas.set_index("Registro",inplace=True)

In [15]:
df_porte = pd.read_csv("../entrada/porte_operadoras.csv",encoding="ISO-8859-1")
df_porte.drop(columns=['beneficiarios'],inplace=True)
df_porte = df_porte[(df_porte['ano'] == 2019) & (df_porte['trimestre'] == 3)]

In [16]:
df_cooperativas_indice_um =  df_cooperativas.merge(df_porte,left_on="Registro",right_on="Registro_ANS")

In [17]:
df_cooperativas_indice_um['porte'].value_counts()

Médio Porte      24
Grande Porte     13
Pequeno Porte    12
Name: porte, dtype: int64

In [18]:
df_porte = df_cooperativas_indice_um[['Registro_ANS',"porte"]]

In [19]:
df_porte.to_csv("../Datasets/idss_alto_nivel.csv",index=False,encoding="ISO-8859-1")